[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tianjianjiang/nlp_data_aug/blob/master/IMDb_baselne.ipynb)

# Prepare

## References
* [fastai/course-v3/nbs/dl1/lesson3-imdb.ipynb](https://nbviewer.jupyter.org/github/fastai/course-v3/blob/master/nbs/dl1/lesson3-imdb.ipynb) (2019-05-03)

learner|bs|lr  |bptt|wd  |drop_mult|`to_fp16(clip)`|fw acc%
--     |-:|--: |--: |--: |--:      |--             |--:
lm     |48|1e-2|70  |0.01|0.3      |No             |34.1371
cf     |48|2e-2|70  |0.01|0.5      |No             |94.3960

* [fastai/fastai/examples/ULMFit.ipynb](https://nbviewer.jupyter.org/github/fastai/fastai/blob/master/examples/ULMFit.ipynb) (2019-06-11)

  > Fine-tuning a forward and backward langauge model to get to 95.4% accuracy on the IMDB movie reviews dataset. This tutorial is done with fastai v1.0.53.

  > The example was run on a Titan RTX (24 GB of RAM) so you will probably need to adjust the batch size accordinly. If you divide it by 2, don't forget to divide the learning rate by 2 as well in the following cells. You can also reduce a little bit the bptt to gain a bit of memory.

learner|bs |lr  |bptt|wd  |drop_mult|`to_fp16(clip)`|fw acc%|bw acc%|avg acc%
--     |--:|--: |--: |--: |--:      |--             |--:    |--:    |--:
lm     |256|2e-2|  80|0.1 |1.0      |Yes; clip=0.1  |34.0075|37.2268|N/A
cf     |128|1e-1| 70†|0.1‡|0.5      |No             |94.9560|94.7400|95.39

    † both cf's used default bptt=70
    ‡ forward cf used default wd=0.01

* [fastai/course-nlp/nn-imdb-more.ipynb](https://nbviewer.jupyter.org/github/fastai/course-nlp/blob/master/nn-imdb-more.ipynb) (2019-06-12)

learner|bs |lr        |bptt|wd  |drop_mult|`to_fp16(clip)`|fw acc%
--     |--:|--:       |--: |--: |--:      |--             |--:
lm     |128|1e-2*bs/48|  70|0.01|1.0      |Yes; clip=None |>27.8265†
cf     |128|2e-2*bs/48|  70|0.01|0.5      |Yes; clip=None |94.8080

    † forward lm's final accuracy wasn't shown

In [0]:
# Ensure GPU spec; T4 is for colab and one can change it for another env.
gpu_list = !nvidia-smi -L
if gpu_list[0].startswith('NVIDIA-SMI has failed'):
  print('Runtime type should be GPU.')
elif not gpu_list[0].startswith('GPU 0: Tesla T4'):
  display(gpu_list)
  print('Please reset all runtimes. We need a Tesla T4 to reproduce the experiments!')
else:
  display(gpu_list)

## Dependency

### Install

In [0]:
# Ensure no surprises from conflict packages.
!pip check

In [0]:
%%capture pip_logs
!pip install -U fastai==1.0.55 ipyexperiments jupyter-console==5.2.0 coverage==4.5.3 coveralls datascience albumentations

In [0]:
colab_vnd = 'application/vnd.colab-display-data+json'
for o in pip_logs.outputs:
  if colab_vnd in o.data and 'pip_warning' in o.data[colab_vnd]:
    o.display()
!pip check

### Import

In [0]:
import gc
import math
from pathlib import Path
import pickle
import random
from shutil import copytree
from typing import Optional, Tuple

import numpy as np
import torch
from google.colab import drive

from fastai import basic_data, basic_train, core
from fastai import *
from fastai.callbacks import CSVLogger
from fastai.core import plt
from fastai.text import *
from fastprogress import fastprogress

from ipyexperiments import *

### Init


In [0]:
# Not set earlier because pip may require a restart.
SESSN_START_T, = !date +%Y%m%dT%H%M

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
# Stylize the plot of `lr_find()`
%config InlineBackend.figure_formats = {'png', 'retina'}
plt.style.use(['dark_background','seaborn-poster','seaborn-deep'])
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.grid.axis'] = 'x'
plt.rcParams['axes.grid.which'] = 'both'
plt.rcParams['grid.alpha'] = 0.5
plt.rcParams['grid.color'] = 'xkcd:lime green'
plt.rcParams['grid.linestyle'] = ':'

In [0]:
# A special treatment for colab to decrease network traffic.
fastprogress.NO_BAR = True
master_bar, progress_bar = fastprogress.force_console_behavior()
basic_train.master_bar, basic_train.progress_bar = master_bar, progress_bar
basic_data.master_bar, basic_data.progress_bar = master_bar, progress_bar
dataclass.master_bar, dataclass.progress_bar = master_bar, progress_bar
text.master_bar, text.progress_bar = master_bar, progress_bar
text.data.master_bar, text.data.progress_bar = master_bar, progress_bar
core.master_bar, core.progress_bar = master_bar, progress_bar

In [0]:
COLAB_CONTENT_DIR_P = Path('/content')
GD_DIR_P = COLAB_CONTENT_DIR_P / 'gdrive'
drive.mount(str(GD_DIR_P), force_remount=True)

In [0]:
BASE_DIR_P = GD_DIR_P / 'My Drive/imdb'
BASE_DIR_P.mkdir(parents=True, exist_ok=True)
DATA_DIR_P = BASE_DIR_P / 'data'
DATA_DIR_P.mkdir(parents=True, exist_ok=True)
MDLS_DIR_P = BASE_DIR_P / 'models'
MDLS_DIR_P.mkdir(parents=True, exist_ok=True)
LOGS_DIR_P = BASE_DIR_P / 'logs'
LOGS_DIR_P.mkdir(parents=True, exist_ok=True)

FASTAI_DATA_DIR_P = Path('/root/.fastai/data')
FASTAI_DATA_DIR_P.mkdir(parents=True, exist_ok=True)
COLAB_DATA_DIR_P = COLAB_CONTENT_DIR_P / 'data'
if not COLAB_DATA_DIR_P.is_symlink():
  COLAB_DATA_DIR_P.symlink_to(FASTAI_DATA_DIR_P)
if (COLAB_CONTENT_DIR_P / 'sample_data').exists():
  !rm -rf /content/sample_data/

IMDB_DATA_IN_COLAB_DIR_P = COLAB_DATA_DIR_P / 'imdb'

# Assign

In [0]:
#@title Hyper-parameters

lm_bs = 128  #@param {type: "number"}
cf_bs = 128  #@param {type: "number"}
bptt = 70  #@param {type: "number"}
moms = (0.8, 0.7)  #@param

#@markdown ---

lm_wd = 0.01  #@param {type: "number"}
cf_wd = 0.01    #@param {type: "number"}
lm_drop_mult = 1.0  #@param {type: "number"}
cf_drop_mult = 0.5  #@param {type: "number"}

FW_LM_DBNCH_FILE_S = f'fw_lm_dbnch-b{lm_bs}-bptt{bptt}.pkl'
BW_LM_DBNCH_FILE_S = f'bw_lm_dbnch-b{lm_bs}-bptt{bptt}.pkl'
FW_CF_DBNCH_FILE_S = f'fw_cf_dbnch-b{cf_bs}-bptt{bptt}.pkl'
BW_CF_DBNCH_FILE_S = f'bw_cf_dbnch-b{cf_bs}-bptt{bptt}.pkl'

VOCAB_FILE_P = DATA_DIR_P / f'imdb_vocab-b{lm_bs}-bptt{bptt}.pkl'

In [0]:
IMDb_CLASSES = ['neg', 'pos']

In [0]:
# Set num_workers to main process since the training set will be shuffled.
n_dbnch_wrkrs = 0

In [0]:
# One seed to rule pseudo-random number generators all.
SEED = 42

# Define

## Random State Fixer

In [0]:
@dataclass
class RandomStateHolder:
  py3_state: Tuple[int, Tuple[int], Optional[float]]
  np_state: Tuple[str, np.ndarray, int, int, float]
  torch_state: torch.ByteTensor
  cuda_states: List[torch.ByteTensor]

In [0]:
def reset_random_states(seed=SEED):
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)  # This implies torch.cuda.manual_seed_all(SEED) now
  # if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)
  torch.backends.cudnn.deterministic = True  # About 15% slower but...
  torch.backends.cudnn.benchmark = False

In [0]:
def get_random_states():
  return RandomStateHolder(
      random.getstate(),
      np.random.get_state(),
      torch.get_rng_state(),
      torch.cuda.get_rng_state_all()
      )

In [0]:
def set_random_states(prngs_states: RandomStateHolder):
  random.setstate(prngs_states.py3_state)
  np.random.set_state(prngs_states.np_state)
  torch.set_rng_state(prngs_states.torch_state)
  torch.cuda.set_rng_state_all(prngs_states.cuda_states)

In [0]:
def save_random_states(prngs_states, file_path):
  with open(file_path, 'wb') as f:
    pickle.dump(prngs_states, f)
    # print(f'Saved prngs_states: {prngs_states}')

In [0]:
def load_random_states(file_path):
  with open(file_path, 'rb') as f:
    prngs_states = pickle.load(f)
    # print(f'Loaded prngs_states: {prngs_states}')
  return prngs_states

## LM-specific Helpers

In [0]:
def build_lm_databunch(data_dir_p, n_workers, bs, bptt, presort=True):
  reset_random_states()
  return (TextList.from_folder(data_dir_p, presort=presort)
          .filter_by_folder(include=['train', 'test', 'unsup'])
          .split_by_rand_pct(
              0.1,
              seed=SEED  # Set the seed again since in theory one can call np.random before this.
          )
          .label_for_lm()
          .databunch(bs=bs, bptt=bptt, num_workers=n_workers))

In [0]:
def load_lm_databunch(
    data_dir_p,
    lm_dbnch_fname_s,
    bs=lm_bs,
    n_workers=n_dbnch_wrkrs,
    bptt=bptt
):
  reset_random_states()
  return load_data(
      data_dir_p, lm_dbnch_fname_s, bs, num_workers=n_workers, bptt=bptt)

In [0]:
def init_lm_learner_with_ulmfit(dbnch, drop_mult, base_path=BASE_DIR_P):
  reset_random_states()
  lm_learn = language_model_learner(dbnch, AWD_LSTM, drop_mult=drop_mult, path=base_path)
  lm_learn = lm_learn.to_fp16()  # 2x faster
  save_random_states(
      get_random_states(),
      DATA_DIR_P/'init_lm_learner_with_ulmfit-prng_states.pkl'
  )
  return lm_learn

In [0]:
def init_lm_cycles(learner, lr, moms, wd, clbks=[], cycle_len=1):
  set_random_states(load_random_states(
      DATA_DIR_P/'init_lm_learner_with_ulmfit-prng_states.pkl'))
  learner.fit_one_cycle(cycle_len, lr, moms=moms, wd=wd, callbacks=clbks)
  save_random_states(
      get_random_states(),
      DATA_DIR_P/'init_lm_cycles-prng_states.pkl'
      )
  return learner

In [0]:
def tune_lm_cycles(learner, lr, moms, wd, clbks=[], cycle_len=10):
  set_random_states(load_random_states(
      DATA_DIR_P/'init_lm_cycles-prng_states.pkl'))
  learner.unfreeze()
  learner.fit_one_cycle(cycle_len, lr, moms=moms, wd=wd, callbacks=clbks)
  save_random_states(
      get_random_states(),
      DATA_DIR_P/'tune_lm_cycles-prng_states.pkl'
      )
  return learner

## CF-specific Helpers

In [0]:
def build_cf_databunch(
    data_dir_p,
    bs,
    vocab,
    tags=IMDb_CLASSES,
    n_workers=n_dbnch_wrkrs,
    presort=True
):
  reset_random_states()
  tl = TextList.from_folder(data_dir_p, vocab=vocab, presort=presort)
  ils = tl.split_by_folder(valid='test')
  lls = ils.label_from_folder(classes=tags)
  return lls.databunch(bs=bs, num_workers=n_workers)

In [0]:
def load_cf_databunch(
    data_dir_p,
    cf_dbnch_fname_s,
    bs=cf_bs,
    n_workers=n_dbnch_wrkrs
):
  reset_random_states()
  return load_data(data_dir_p, cf_dbnch_fname_s, bs, num_workers=n_dbnch_wrkrs)

In [0]:
def init_cf_learner_with_encoder(enc_name, dbnch, drop_mult, bptt=bptt, base_path=BASE_DIR_P):
  reset_random_states()
  cf_learn = text_classifier_learner(dbnch, AWD_LSTM, drop_mult=drop_mult, bptt=bptt, path=base_path, pretrained=False)
  cf_learn = cf_learn.to_fp16()
  cf_learn.load_encoder(enc_name)
  save_random_states(
      get_random_states(),
      DATA_DIR_P/f'init_cf_learner_with_encoder-prng_states.pkl'
  )
  return cf_learn

In [0]:
def init_cf_cycles(learner, lr, moms, wd, clbks, cycle_len=1):
  set_random_states(load_random_states(
      DATA_DIR_P/'init_cf_learner_with_encoder-prng_states.pkl'))
  learner.fit_one_cycle(cycle_len, lr, moms=moms, wd=wd, callbacks=clbks)
  save_random_states(
      get_random_states(),
      DATA_DIR_P/'init_cf_cycles-prng_states.pkl'
      )

  learner.path = COLAB_DATA_DIR_P
  tmp_name = f'init_cf_tmp-{lr}'
  tmp_p = learner.save(tmp_name, return_path=True, with_opt=True)
  learner.load(tmp_name, purge=True, with_opt=True)
  tmp_p.unlink()
  learner.path = BASE_DIR_P
  torch.cuda.empty_cache()

  return learner

In [0]:
def tune_cf_cycles(
    learner,
    base_lr,
    moms,
    wd,
    clbks_tuple,
    cycle_len_tuple=(1,1,2),
    freeze_steps=(-2,-3,None),
    lr_decays=(2,2*2,2*2*5)
):
  prev_prng_states_pkl_path = DATA_DIR_P/'init_cf_cycles-prng_states.pkl'
  cycle_id = 0

  for cycle_len, freeze_step, lr_decay, clbks in zip(
      cycle_len_tuple, freeze_steps, lr_decays, clbks_tuple):
    if freeze_step is not None:
      learner.freeze_to(freeze_step)
    else:
      learner.unfreeze()
    lr = base_lr / lr_decay

    set_random_states(load_random_states(prev_prng_states_pkl_path))
    learner.fit_one_cycle(cycle_len, slice(lr/(2.6**4),lr), moms=moms, wd=wd, callbacks=clbks)
    cycle_id += 1
    prng_states_pkl_path = DATA_DIR_P/f'tune_cf_cycles-{cycle_id}-prng_states.pkl'
    save_random_states(get_random_states(), prng_states_pkl_path)
    prev_prng_states_pkl_path = prng_states_pkl_path
    
    learner.path = COLAB_DATA_DIR_P
    tmp_name = f'tune_cf_tmp-{lr}'
    tmp_p = learner.save(tmp_name, return_path=True, with_opt=True)
    learner.load(tmp_name, purge=True, with_opt=True)
    tmp_p.unlink()
    learner.path = BASE_DIR_P
    torch.cuda.empty_cache()

  return learner

# Fit

## Forward LM

### Process Data Once

In [0]:
imdb_data_downloaded_dir_p = untar_data(URLs.IMDB, dest=COLAB_DATA_DIR_P)
assert imdb_data_downloaded_dir_p == IMDB_DATA_IN_COLAB_DIR_P

In [0]:
if (DATA_DIR_P / FW_LM_DBNCH_FILE_S).exists():
  print(f'Loading {DATA_DIR_P/FW_LM_DBNCH_FILE_S}.....')
  fw_lm_dbnch = load_lm_databunch(DATA_DIR_P, FW_LM_DBNCH_FILE_S, lm_bs, n_dbnch_wrkrs, bptt)
else:
  print(f'Building {FW_LM_DBNCH_FILE_S}......')
  fw_lm_dbnch = build_lm_databunch(IMDB_DATA_IN_COLAB_DIR_P, n_dbnch_wrkrs, lm_bs, bptt)
# fw_lm_dbnch.show_batch()

In [0]:
if not VOCAB_FILE_P.exists():
  print(f'Building {VOCAB_FILE_P}......')
  fw_lm_dbnch.vocab.save(VOCAB_FILE_P)
print(f'Loading {VOCAB_FILE_P}......')
IMDB_VOC = Vocab.load(VOCAB_FILE_P)

### Use Persistent Path

In [0]:
# Save the databunch to a non-voatile path (e.g.: GDrive).
if not (DATA_DIR_P / FW_LM_DBNCH_FILE_S).exists():
  print(f'Saving {DATA_DIR_P/FW_LM_DBNCH_FILE_S}......')
  fw_lm_dbnch.save(DATA_DIR_P / FW_LM_DBNCH_FILE_S)

In [0]:
# The batch should look the same if the above efforts keep the reproducibility.
# fw_lm_dbnch.show_batch()

### Find Learning Rate

In [0]:
assert fw_lm_dbnch.train_dl.batch_size == lm_bs
lm_epoch_sz = math.ceil(len(fw_lm_dbnch.train_ds) / lm_bs)
lr_find_num_it = math.ceil(lm_epoch_sz*0.3)  # `OneCycleScheduler`'s `pct_start=0.3`
print(f'lm_epoch_sz\t= {lm_epoch_sz}\nlr_find_num_it\t= {lr_find_num_it}')

In [0]:
lr_find_scope = IPyExperimentsPytorch(cl_enable=False)
fw_lm_dbnch = load_lm_databunch(DATA_DIR_P, FW_LM_DBNCH_FILE_S, lm_bs, n_dbnch_wrkrs, bptt)
fw_lm_learn = init_lm_learner_with_ulmfit(fw_lm_dbnch, lm_drop_mult, COLAB_DATA_DIR_P)
fw_lm_learn.lr_find(start_lr=1e-3, end_lr=1e-1, num_it=lr_find_num_it, wd=lm_wd)

In [0]:
%%capture lr_find_log
fw_lm_learn.recorder.plot(suggestion=True)

In [0]:
min(fw_lm_learn.recorder.losses)

In [0]:
lr_find_log()

In [0]:
((found_lr_desc, found_lr_val_str),
 (min_loss_desc, min_loss_val_str)
 ) = [line.split(': ') for line in lr_find_log.stdout.split('\n') if line]

In [0]:
lr_find_scope.keep_var_names('found_lr_val_str')
del lr_find_scope
gc.collect()

### Init-fit

In [0]:
found_lm_lr = round(float(found_lr_val_str), 4)
referred_lm_lr = 1e-2 * 128 / 48
lm_lr = 1e-3 * 25  # 25 is the div_factor of slanted triangular lr scheduler
print(f'found lm_lr     ={found_lm_lr}\n'
      f'referred lm_lr  ={referred_lm_lr}\n'
      f'designated lm_lr={lm_lr}')
init_fw_lm_name = f'init_fw_lm-b{lm_bs}-bptt{bptt}-lr{lm_lr}'

In [0]:
init_lm_scope = IPyExperimentsPytorch(cl_enable=False)
fw_lm_dbnch = load_lm_databunch(DATA_DIR_P, FW_LM_DBNCH_FILE_S, lm_bs, n_dbnch_wrkrs, bptt)
fw_lm_learn = init_lm_learner_with_ulmfit(fw_lm_dbnch, lm_drop_mult)
init_fw_lm_log_p = LOGS_DIR_P / f'{SESSN_START_T}-{init_fw_lm_name}'  # w/o .csv
init_fw_lm_clbks = [CSVLogger(fw_lm_learn, init_fw_lm_log_p, append=True)]

In [0]:
print(init_fw_lm_name)
fw_lm_learn = init_lm_cycles(fw_lm_learn, lm_lr, moms, lm_wd, init_fw_lm_clbks)
# fw_lm_learn.csv_logger.read_logged_file()
fw_lm_learn.save(init_fw_lm_name, with_opt=True)
# (fw_lm_learn.path/fw_lm_learn.model_dir).ls()

In [0]:
del init_lm_scope
gc.collect()

### Fine-tune

In [0]:
tune_lm_lr = lm_lr / 10
tune_fw_lm_name = f'tune_fw_lm-b{lm_bs}-bptt{bptt}-lr{tune_lm_lr}'
fw_enc_name = f'fw_enc-b{lm_bs}-bptt{bptt}-lr{lm_lr}'

In [0]:
tune_lm_scope = IPyExperimentsPytorch(cl_enable=False)
fw_lm_dbnch = load_lm_databunch(DATA_DIR_P, FW_LM_DBNCH_FILE_S, lm_bs, n_dbnch_wrkrs, bptt)
fw_lm_learn = init_lm_learner_with_ulmfit(fw_lm_dbnch, lm_drop_mult)
fw_lm_learn = fw_lm_learn.load(init_fw_lm_name, purge=True, with_opt=True)
tune_fw_lm_log_p = LOGS_DIR_P / f'{SESSN_START_T}-{tune_fw_lm_name}'
tune_fw_lm_clbks = [CSVLogger(fw_lm_learn, tune_fw_lm_log_p, append=True)]

In [0]:
print(tune_fw_lm_name)
fw_lm_learn = tune_lm_cycles(fw_lm_learn, tune_lm_lr, moms, lm_wd, tune_fw_lm_clbks)
fw_lm_learn.save(tune_fw_lm_name, with_opt=True)
fw_lm_learn.save_encoder(fw_enc_name)
# (fw_lm_learn.path/fw_learn_lm.model_dir).ls()

In [0]:
del tune_lm_scope
gc.collect()

## Forward CF

In [0]:
if (DATA_DIR_P / FW_CF_DBNCH_FILE_S).exists():
  print(f'Loading {DATA_DIR_P/FW_CF_DBNCH_FILE_S}.....')
  fw_cf_dbnch = load_cf_databunch(DATA_DIR_P, FW_CF_DBNCH_FILE_S, cf_bs, n_dbnch_wrkrs)
else:
  print(f'Building {FW_CF_DBNCH_FILE_S}......')
  fw_cf_dbnch = build_cf_databunch(IMDB_DATA_IN_COLAB_DIR_P, cf_bs, IMDB_VOC, n_workers=n_dbnch_wrkrs)
# fw_cf_dbnch.show_batch()

In [0]:
# Save the databunch to a non-voatile path (e.g.: GDrive).
if not (DATA_DIR_P / FW_CF_DBNCH_FILE_S).exists():
  print(f'Saving {DATA_DIR_P/FW_LM_DBNCH_FILE_S}......')
  fw_cf_dbnch.save(DATA_DIR_P / FW_CF_DBNCH_FILE_S)

In [0]:
cf_lr = lm_lr * 2
init_fw_cf_name = f'init_fw_cf-b{cf_bs}-bptt{bptt}-lr{cf_lr}'

In [0]:
init_cf_scope = IPyExperimentsPytorch(cl_enable=False)
fw_cf_dbnch = load_cf_databunch(DATA_DIR_P, FW_CF_DBNCH_FILE_S, cf_bs, n_dbnch_wrkrs)
fw_cf_learn = init_cf_learner_with_encoder(fw_enc_name, fw_cf_dbnch, cf_drop_mult, bptt)
init_fw_cf_log_p = LOGS_DIR_P / f'{SESSN_START_T}-{init_fw_cf_name}'  # w/o .csv
init_fw_cf_clbks = [CSVLogger(fw_cf_learn, init_fw_cf_log_p, append=True)]

In [0]:
print(init_fw_cf_name)
fw_cf_learn = init_cf_cycles(fw_cf_learn, cf_lr, moms, cf_wd, init_fw_cf_clbks)
fw_cf_learn.save(init_fw_cf_name, with_opt=True)

In [0]:
del init_cf_scope
gc.collect()

In [0]:
tune_fw_cf_name = f'tune_fw_cf-b{cf_bs}-bptt{bptt}-lr{cf_lr}'

In [0]:
tune_cf_scope = IPyExperimentsPytorch(cl_enable=False)
fw_cf_dbnch = load_cf_databunch(DATA_DIR_P, FW_CF_DBNCH_FILE_S, cf_bs, n_dbnch_wrkrs)
fw_cf_learn = init_cf_learner_with_encoder(fw_enc_name, fw_cf_dbnch, cf_drop_mult, bptt)
fw_cf_learn = fw_cf_learn.load(init_fw_cf_name, purge=True, with_opt=True)
tune_fw_cf_clbks_tuple = (
    [CSVLogger(
        fw_cf_learn,
        LOGS_DIR_P / f'{SESSN_START_T}-{tune_fw_cf_name}-p{period}',
        append=True)]
    for period in range(1,4)
)

In [0]:
print(tune_fw_cf_name)
fw_cf_learn = tune_cf_cycles(fw_cf_learn, cf_lr, moms, cf_wd, tune_fw_cf_clbks_tuple)
fw_cf_learn.save(tune_fw_cf_name, with_opt=True)
fw_cf_learn.export(MDLS_DIR_P / f'export-{tune_fw_cf_name}')

In [0]:
del tune_cf_scope
gc.collect()